# Visualization and Table Builder for Operational DSM2 Analysis

This is a Jupyter Notebook it works by clicking on a cell and pressing shift+enter to execute the code in the cell. All the code in this notebook should be execute to produce outputs.

In [ ]:
# imported libraries required for running this notebook
import os
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import panel as pn
import geoviews as gv
import holoviews as hv
import geopandas as gpd
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import ks_2samp
gv.extension("bokeh", "plotly")

## USER REQUIRE INPUTS CELL

The cell below contains variables for all the required user inputs for running this notebook from one week to the next. These user inputs should be modified by the user.

In [ ]:
# User Required Inputs
# 0.) Working directory for the DSM2 study
primary_work_dir = r""
# 1.) Channels Geojson Location, default = ".\channels.geojson"
channels_pathname = ".\channels.geojson"
# 2.) Output runid folder from the post-processing tool
# should contain VarKS.csv and VarTotal.csv
runid_output_folder = ""
# 3.) Action start date, format YYYY-MM-DD
action_start = ""
# 4.) Action end date, format YYYY-MM-DD
action_end = ""
# 5.) *.html Panel Visualization output file name
panel_html_vis_output_filename = "dsm2.html"
# 6.) Output folder for the *.png extend graphs
graph_png_output_foldername = "extend_graphs"
# 7.) Output folder for the *.csv tables
tables_csv_output_foldername = "extend_tables"

In [ ]:
runid_output_dir = os.path.join(primary_work_dir, runid_output_folder)
panel_html_vis_output_pathname = os.path.join(primary_work_dir,
                                              panel_html_vis_output_filename)
graph_png_output_pathname = os.path.join(primary_work_dir,
                                         graph_png_output_foldername)
tables_csv_output_pathname = os.path.join(primary_work_dir,
                                          tables_csv_output_foldername)
if not os.path.exists(graph_png_output_pathname):
    os.mkdir(graph_png_output_pathname)
if not os.path.exists(tables_csv_output_pathname):
    os.mkdir(tables_csv_output_pathname)
    


### Starting here code is executed.

This cell does the following:  
1.) Read-in the channels geojson file as a geopandas dataframe.  
2.) Read-in VarKS.csv as a pandas dataframe from runid_output_dir.  
3.) Read-in VarTotal.csv as a pandas dataframe from runid_output_dir.  
4.) Convert the VarTotal dataframe column 'datetime' as a pandas datetime type.  
5.) Select VarTotal dataframe based on the timeframe from action_start to action_end.  

In [ ]:
channels = gpd.read_file(channels_pathname)
varks_df = pd.read_csv(os.path.join(runid_output_dir, "VarKS.csv"),
                       sep=",", infer_datetime_format=True,
                       parse_dates=True, header=0, index_col=0)
vartotal_df = pd.read_csv(os.path.join(runid_output_dir, "VarTotal.csv"),
                          sep=",", infer_datetime_format=True,
                          parse_dates=True, header=0, index_col=0)
vartotal_df["datetime"] = pd.to_datetime(vartotal_df["datetime"])
select_datetime_range = pd.date_range(start=action_start,
                                      end=action_end, freq="15min")
vartotal_df = vartotal_df.loc[vartotal_df["datetime"]
                              .isin(select_datetime_range)]

Displays the outputs from above for the user to check.

In [ ]:
display(channels)
display(varks_df)
display(vartotal_df)

Creates the identifiers for each scenario and variable pair.  
For example "OMR-5000 FLOW"

In [ ]:
map_ks_options = []
for scenario in varks_df["scenario1"].unique():
    for variable in varks_df["variable"].unique():
        map_ks_options.append("{} {}".format(scenario, variable))
test_map_value = map_ks_options[0]
display(map_ks_options)

Creates a list for the variable identifiers and the important channels from the DSM2 analysis. The full important channels are set as channel_lst, while the most important channels are set as critical_channel_lst. The channels were identified by BDO.

In [ ]:
variable_lst = ["FLOW", "VEL"]
channel_lst = [6, 21, 24, 49, 54, 81, 94, 107, 117, 124, 148, 160,
               173, 214, 227, 310, 421, 422, 423, 434]
critical_channel_lst = [6, 21, 49, 81, 94, 107, 124, 148, 160, 434]
channel_lst = [int(x) for x in channel_lst]
critical_channel_lst = [int(x) for x in critical_channel_lst]

Creates the two dropdown widgets for the *.html* Panel Visualization.

In [ ]:
variable_dropdown = pn.widgets.Select(name="variable_dropdown",
                                      options=variable_lst)
channel_dropdown = pn.widgets.Select(name="channel_dropdown",
                                     options=channel_lst)

Function for building each map in the *.html* Panel Visualization.  
This function is called by the Panel Builder for each map based on the map_ks_options variable.

In [ ]:
def build_map(value):
    global channels
    global varks_df
    ichannels = channels.copy()
    base_map = gv.tile_sources.CartoLight.opts(width=950, height=600)
    select_ks_stat = varks_df.loc[(varks_df["scenario1"] == 
                                   value.split(" ")[0]) &
                                  (varks_df['variable'] == 
                                   value.split(" ")[1])]
    select_ks_stat = select_ks_stat[["channel", "ks_stat"]].set_index('channel')
    input_df = ichannels.set_index("channel_nu").join(select_ks_stat)
    input_df[["ks_stat"]] = input_df[["ks_stat"]].fillna(0)
    delta = gv.Contours(input_df,
                        vdims=["ks_stat", "channel_nu"]).opts(cmap="turbo_r",
                                                              tools=["hover"],
                                                              width=600,
                                                              height=600,
                                                              line_width=3,
                                                              colorbar=True,
                                                              show_legend=False,
                                                              clim=(0, 1))
    total_map = base_map*delta
    return total_map

Function for building each plotly graph in the *.html* Panel Visualization.  
This function is called by the Panel Builder for each variable, channel pair.

In [ ]:
def build_mapgraph(variable, channel):
    global vartotal_df
    global varks_df
    df = vartotal_df
    df_ks = varks_df
    DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                           'rgb(188, 189, 34)', 'rgb(140, 86, 75)',
                           'rgb(148, 103, 189)', 'rgb(127, 127, 127)',
                           'rgb(227, 119, 194)', 'rgb(23, 190, 207)',
                           'rgb(44, 160, 44)', 'rgb(214, 39, 40)']    
    df_select = df.loc[(df["variable"] == variable) &
                       (df["channel"] == channel)
                       ]
    
    trace_lst = []
    ks_lst = []
    for indx, scenario in enumerate(df_select["scenario"].unique()):
        data_arr = (df_select
                    .loc[(df_select["scenario"] == scenario)]["value"]
                    .to_numpy(dtype=np.float32))
        ecdf_obj = ECDF(data_arr)
        trace = go.Scatter(x=ecdf_obj.x, y=ecdf_obj.y,
                           mode="lines", name=scenario,
                           line=dict(color=DEFAULT_PLOTLY_COLORS[indx]))
        trace_lst.append(trace)
        if not "Baseline" in scenario:
            ks_value = (df_ks.loc[(df_ks["scenario1"] == scenario) &
                                 (df_ks["variable"] == variable) &
                                 (df_ks["channel"] == channel)]["ks_stat"]
                        .values.tolist())
            ks_lst.append("{}: {}".format(scenario, ks_value[0]))
    xt_dict = {"FLOW": ["Flow", "CFS"], "VEL": ["Velocity", "FT/S"]}
    ks_stat = "|".join(ks_lst)
    KS_annotation = [go.layout.Annotation(x=0, y=1.10,
                                          xref="paper",
                                          yref="paper",
                                          showarrow=False,
                                          text="Kolmogorov-Smirnov " +
                                          "Distance: {}"
                                          .format(ks_stat),
                                         font=dict(size=16))]
    xt_var = xt_dict.get(variable)
    xt = "{} in {}".format(xt_var[0], xt_var[1])
    yt = "Fraction of Data"
    layout = go.Layout(autosize=False, width=800, height=600,
                       annotations=KS_annotation,
                       legend=dict(font=dict(size=16)),
                       legend_orientation="v",
                       xaxis=dict(title=dict(text=xt, font=dict(size=24))),
                       yaxis=dict(title=dict(text=yt, font=dict(size=24))))
    ecdf_fig = go.Figure(data=trace_lst,
                         layout=layout)
    return ecdf_fig

Displays sample visualizations of the mapgraph and the map for the user to check.

In [ ]:
display(build_mapgraph("FLOW", 214))
display(build_map(test_map_value))

Creates a reactive panel variable for the graph objects in the *.html* Panel Visualization.  
This effectively creates the graphs for each of the dropdown options.

In [ ]:
@pn.depends(variable_dropdown, channel_dropdown)
def reactive_graph(variable_dropdown, channel_dropdown):
    return build_mapgraph(variable_dropdown, channel_dropdown)

Creates the panel tabs for each of the map objeccts in the *.html* Panel Visualization.

In [ ]:
tabs = pn.Tabs()
for opt in map_ks_options:
    map_obj = build_map(opt)
    tabs.append((opt, map_obj))

Gspec is the Panel Builder object that becomes saved as the *.html* Panel Visualization.

In [ ]:
gspec = pn.GridSpec(sizing_mode="stretch_both", max_height=800, max_width=800)

In [ ]:
gspec[0:2, 0] = pn.Column(tabs)

In [ ]:
gspec[0:2, 1] = pn.Column(pn.Row(variable_dropdown,
                                 channel_dropdown),
                          reactive_graph)

In [ ]:
gspec.save(panel_html_vis_output_pathname, embed=True, max_opts=10000)

Function for writing out the extended graphs for input into reporting.

In [ ]:
def MakeExtendGraphs(channel_lst, variable_lst, output_dir):
    for v in variable_lst:
        for c in channel_lst:
            print(v, c)
            fig = build_mapgraph(v, c)
            output_pathname = os.path.join(output_dir, "{}_{}_ecdf.png".format(v, c))
            pio.write_image(fig, output_pathname)
    return 0

In [ ]:
MakeExtendGraphs(channel_lst, variable_lst, graph_png_output_pathname)

### Create Additional Statistics and KS-Stat Tables

Function for building the additional statistic tables for min, max, mean, and percent positive.

In [ ]:
def MakeStatTable(vartotal_df, channels=channel_lst):
    df = vartotal_df.copy()
    df_channel = df.loc[(df.channel.isin(channels))]
    grouper = df_channel.groupby(["scenario", "variable", "channel"])
    df_describe = grouper.describe()
    df_describe.columns = df_describe.columns.droplevel(0)
    df_describe = df_describe.sort_index(level=["variable", "channel"])
    df_stats = df_describe[["min", "max", "mean"]]
    df_percent = grouper.apply(lambda x: ((x["value"]>0).sum()
                               / x["value"].count())*100)
    df_percent = df_percent.sort_index(level=["variable", "channel"])
    df_final = pd.concat([df_stats, df_percent], axis=1)
    df_final = df_final.rename(columns={0: "% positive"})
    df_final = df_final.round(2)
    df_final = df_final.unstack("variable")  
    df_final = df_final.swaplevel(0, -1, axis=1)
    df_final.columns.set_levels(["Flow", "Velocity"], level=0, inplace=True)
    df_final.columns.set_levels(["Minimum Flow", "Maximum Flow", "Mean Flow", "% Positive Flow",
                                 "Minimum Velocity", "Maximum Velocity", "Mean Velocity",
                                 "% Positive Velocity"],level=1,inplace=True)
    df_final = df_final.sort_index(axis=1, level=0, sort_remaining=False)
    df_final = df_final.sort_index(axis=0, level=1, sort_remaining=False)
    df_final.columns.names = (None, None)  
    df_final.index.names = (None, "DSM2 Channel")
    return df_final

Create Additional Statistic Tables for the channel_lst variable.

In [ ]:
df_stat = MakeStatTable(vartotal_df, channels=channel_lst)

In [ ]:
display(df_stat)

Create Additional Stastistic Tables for the critical_channel_lst variable.

In [ ]:
df_cstat = MakeStatTable(vartotal_df,
                         channels=critical_channel_lst)

In [ ]:
display(df_cstat)

Function for building the additional KS-Stat tables.

In [ ]:
def MakeKSTable(varks_df, channels=channel_lst):
    df = varks_df.copy()
    df_channels = df.loc[(df.channel.isin(channels))]
    df_channels = df_channels.set_index(["scenario0", "scenario1",
                                         "variable", "channel"])
    df_ks = df_channels.unstack(["scenario0", "scenario1"])
    df_ks = df_ks.sort_index(level=["variable", "channel"])
    df_ks.columns = df_ks.columns.droplevel(0)
    df_ks = df_ks.unstack("variable")
    
    df_ks = df_ks.swaplevel(0, -1, axis=1)
    df_ks = df_ks.swaplevel(-1, 1, axis=1)

    df_ks.columns.names = (None, None, None)
    df_ks.index.names = ["DSM2 Channel"]
    df_ks = df_ks.sort_index(axis=1, level=0, sort_remaining=False)
    df_ks.columns.set_levels(["Flow", "Velocity"], level=0, inplace=True)
    df_ks = df_ks.round(4)
    return df_ks


Create KS-Stat Tables for the channel_lst variable.

In [ ]:
df_ks = MakeKSTable(varks_df, channels=channel_lst)

In [ ]:
display(df_ks)

Create KS-Stat Tables for the critical_channel_lst variable.

In [ ]:
df_cks = MakeKSTable(varks_df, channels=critical_channel_lst)

In [ ]:
display(df_cks)

Write out the Additional Statistics and KS-Stat Tables to the tables_csv_output_pathname directory.

In [ ]:
df_write_dict = {'critical__statistics': df_cstat,
                 'critical_ks': df_cks,
                 'statistics': df_stat,
                 'ks': df_ks}


def WriteExtraTablesCSV(df_write_dict, output_dir):
    for k, v in df_write_dict.items():
        print(k)
        pathname = os.path.join(output_dir, "{}.csv".format(k))
        print(pathname)
        v.to_csv(pathname, sep=',')
    return 0

In [ ]:
WriteExtraTablesCSV(df_write_dict, tables_csv_output_pathname)

End.